# Auto Scraper


In [2]:
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import statistics
import undetected_chromedriver as uc

In [20]:
def initiate (enlace=0):
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = uc.Chrome()
    driver.implicitly_wait(5)
    
    driver.get('https://www.fotocasa.es/es/')
    time.sleep(3)
    driver.maximize_window()
    time.sleep(2)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar y cerrar')]"))).click()
    time.sleep(3)
    return driver
    

In [21]:
#driver= initiate()

In [22]:
def buscar_distrito(distrito, driver):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomInput-input sui-AtomInput-input-size-m'.replace(' ', '.')))).click()
    time.sleep(3)
    
    buscar= driver.find_element(by=By.CLASS_NAME,value="sui-AtomInput-input sui-AtomInput-input-size-m".replace(' ', '.'))
    buscar.send_keys(distrito)
    time.sleep(3)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Buscar')]"))).click()

In [23]:
#buscar_distrito('san esteban leon',driver)

In [24]:
def scroll_down(driver):  
    elem = driver.find_element(By.TAG_NAME,value="body")
    no_of_pagedowns = 25
    time.sleep(3)
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        no_of_pagedowns-=1
        
    time.sleep(3)
    element = driver.find_element(By.CLASS_NAME,'sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded'.replace(' ', '.'))
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()   

In [25]:
#scroll_down(driver)

In [26]:
def get_html(driver):
    html_txt=driver.page_source
    soup=BeautifulSoup(html_txt)
    viviendas= soup.find('section',class_='re-SearchResult')
    return viviendas

In [27]:
#viviendas=get_html(driver)

In [28]:
def get_data(viviendas,distrito):
    try:
        vivienda_str_list=[]

        for vivienda in viviendas:     
            vivienda_str_len=len(str(vivienda))
            vivienda_str_list.append(vivienda_str_len)

        mean_len=statistics.mean(vivienda_str_list)*0.80
        print('Mean length value of the html:',mean_len)
        
        precios = get_precios(viviendas, mean_len)
        titulos = get_titulos(viviendas, mean_len)
        habitaciones = get_habitaciones(viviendas, mean_len)
        baños = get_baños(viviendas, mean_len)
        superficie = get_superficie(viviendas, mean_len)
        ascensor = get_ascensor (viviendas, mean_len)
        planta = get_planta (viviendas, mean_len)
        link = get_link (viviendas, mean_len)
        fechas = get_publicacion(viviendas, mean_len)
        df=pd.DataFrame(list(zip(titulos,precios,habitaciones,planta,superficie,baños,ascensor,fechas ,link))).drop_duplicates()
        df=df.rename(columns={0:"Título",1:"Precio",2:"Habitaciones",3:"Planta",4:"Superficie",5:"Baños",6:"Ascensor",7:"Fecha Publicación",8:"Links"})
        ciudades = ['Leon', 'Alicante']
        df['Barrio']=distrito
        for ciudad in ciudades:
            if ciudad in distrito: 
                df['Ciudad']=ciudad
    except:
        df=pd.DataFrame([])
    return df
    
    
def get_precios(viviendas, mean_len):   
    lista_precios=[]
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass
        else:
            try:
                precio=vivienda.find('span',class_='re-CardPrice').getText()
                lista_precios.append(precio)
            except AttributeError:
                pass
            
    return lista_precios


def get_titulos(viviendas, mean_len):
    lista_titulos=[]
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass
        else: 
            titulos=vivienda.find('span',class_='re-CardTitle').getText()
            lista_titulos.append(titulos)
    return lista_titulos

def get_habitaciones(viviendas, mean_len):
    lista_habitaciones=[]
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        hab_pos=re.search(r'habs.',vivienda_str)
        hab_pos_indi=re.search(r'hab.',vivienda_str)

        if len(str(vivienda))<mean_len:
            pass

        elif hab_pos:
            hab_pos_text=hab_pos.start()
            hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
            lista_habitaciones.append(hab)
        else:
            hab_pos_text=hab_pos_indi.start()
            hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
            lista_habitaciones.append(hab)       

    return lista_habitaciones

def get_baños(viviendas, mean_len):
    lista_baños=[]
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)


        if len(str(vivienda))<mean_len:
            pass

        else:

            baño_pos=re.search(r'(baño|Baño)',vivienda_str)
            baño_pos_text=baño_pos.start()
            b=vivienda_str[baño_pos_text-2:baño_pos_text-1]
            lista_baños.append(b)
 
    return lista_baños
    
def get_superficie (viviendas, mean_len):
    lista_superficie=[]
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        if len(str(vivienda))<mean_len:
            pass
        else:
            superficie_pos=re.search(r'([0-9] [m])',vivienda_str)
            superficie_pos_text=superficie_pos.start()
            s=vivienda_str[superficie_pos_text-2:superficie_pos_text+3]

            if 'm' not in s:
                lista_superficie.append(None)
            else:
                lista_superficie.append(re.findall(r'\d+',s)[0])

    return lista_superficie

def get_ascensor (viviendas, mean_len):
    lista_ascensor = []
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        keyword='ascensor'

        if len(str(vivienda))<mean_len:
            pass

        elif keyword in vivienda_str:
            lista_ascensor.append('SI')

        else:
            lista_ascensor.append('NO')
    return lista_ascensor

def get_planta (viviendas, mean_len):
    lista_planta = []
    for vivienda in viviendas:     
        vivienda_str=str(vivienda)
        planta='Planta'
        planta_baja='planta'

        if len(str(vivienda))<mean_len:
            pass

        elif planta in vivienda_str:
            planta_pos=re.search(r'(Planta)',vivienda_str)
            planta_pos_text=planta_pos.start()
            p=vivienda_str[planta_pos_text-3:planta_pos_text-2]
            lista_planta.append(p)        

        elif planta_baja in vivienda_str:
            planta_pos_baja=re.search(r'(planta)',vivienda_str)
            planta_pos_text_baja=planta_pos_baja.start()
            lista_planta.append(0)        

        else:
            lista_planta.append(None)
        
    return lista_planta
    
def get_link(viviendas, mean_len):
    lista_links = []
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass
        else:
            links = vivienda.find('a',href=True)
            href = links['href']
            fotocasa = 'https://www.fotocasa.es/'
            if '/es/comprar/vivienda/' in href:
                lista_links.append(fotocasa+href)
                
    return lista_links

def get_publicacion (viviendas, mean_len):
    lista_fechas = []
    for vivienda in viviendas:     
        if len(str(vivienda))<mean_len:
            pass

        else:
            fecha=vivienda.find('span',class_='re-CardTimeAgo').getText()
            if re.search(r'\d+', fecha):
                fecha_int=int(re.search(r'\d+', fecha).group())
            else:
                fecha_int=0
            lista_fechas.append(fecha_int)
            
    return lista_fechas


In [29]:
#df=get_data(viviendas)


In [30]:
def get_max_value(driver):
    resultSet = driver.find_element(By.XPATH,"//ul[@class='sui-MoleculePagination']")
    list_pages=[]
    all_li = resultSet.find_elements(By.TAG_NAME,"li")
    for li in all_li:
        text = li.text
        try:
            text=int(text)
            list_pages.append(text)
        except:
            pass
        
    max_value=max(list_pages)
    return max_value

In [31]:
#max_value=get_max_value(driver)

In [44]:
def next_page(driver):
    try:
        time.sleep(3)
        buttons = driver.find_element(By.XPATH,"//ul/li[@class='sui-MoleculePagination-item'][last()]/a[@class='sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded']")
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(buttons)).click()
        time.sleep(7)
    except:
        time.sleep(3)
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='ab-close-button']"))).click()
        time.sleep(3)
        buttons = driver.find_element(By.XPATH,"//ul/li[@class='sui-MoleculePagination-item'][last()]/a[@class='sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded']")
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(buttons)).click()
        time.sleep(7)

In [33]:
#next_page(driver)

In [34]:
#scroll_down(driver)

In [35]:
#viviendas=get_html(driver)

In [36]:
#n_df=get_data(viviendas)

In [37]:
#df = pd.concat([df,n_df],axis=0)

In [38]:
#df.shape

In [42]:
def batch(distrito):
    driver = initiate()
    buscar_distrito(distrito,driver)
    scroll_down(driver)
    max_value=get_max_value(driver)
    viviendas = get_html(driver)
    df=get_data(viviendas,distrito)
    for i in range(max_value-1):
        next_page(driver)
        scroll_down(driver)
        viviendas=get_html(driver)
        n_df=get_data(viviendas,distrito)
        print(n_df.shape)
        df = pd.concat([df,n_df],axis=0)
        print(df.shape)
        
    return df


In [48]:
if __name__ == '__main__':
    
    busquedas = ['San Esteban Leon Capital', 'La Chantría Leon Capital', 'El Ejido Leon Capital', 'La Palomera Leon Capital', 'Carolinas Bajas Alicante', 'Campoamor Alicante','Pla del Bon Repos - La Goteta Alicante']
    df = pd.DataFrame()
    for busqueda in busquedas:
        data = batch(busqueda)
        df = pd.concat([df, data],axis=0)
        print(df.shape)


Mean length value of the html: 6255.660606060606
Mean length value of the html: 5724.751515151515
(0, 0)
(0, 0)
Mean length value of the html: 4958.400000000001
(30, 11)
(30, 11)
Mean length value of the html: 5254.545454545455
(28, 11)
(58, 11)
Mean length value of the html: 5511.418181818182
(29, 11)
(87, 11)
Mean length value of the html: 5420.048484848485
(0, 0)
(87, 11)
Mean length value of the html: 5592.872727272727
(27, 11)
(114, 11)
Mean length value of the html: 5548.848484848485
(0, 0)
(114, 11)
Mean length value of the html: 5610.666666666667
(0, 0)
(114, 11)
Mean length value of the html: 5773.648484848485
(29, 11)
(143, 11)
Mean length value of the html: 5735.272727272728
(28, 11)
(171, 11)
Mean length value of the html: 5206.424242424243
(29, 11)
(200, 11)
Mean length value of the html: 5071.466666666667
(0, 0)
(200, 11)
Mean length value of the html: 3985.92
(0, 0)
(200, 11)
(200, 11)
Mean length value of the html: 7379.882352941177
Mean length value of the html: 6362.3

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.77)
Stacktrace:
0   c048eed55a3e6615_chromedriver       0x0000000104686138 c048eed55a3e6615_chromedriver + 4923704
1   c048eed55a3e6615_chromedriver       0x00000001045fe9d3 c048eed55a3e6615_chromedriver + 4368851
2   c048eed55a3e6615_chromedriver       0x0000000104247787 c048eed55a3e6615_chromedriver + 472967
3   c048eed55a3e6615_chromedriver       0x000000010421dc15 c048eed55a3e6615_chromedriver + 302101
4   c048eed55a3e6615_chromedriver       0x00000001042ba4cf c048eed55a3e6615_chromedriver + 943311
5   c048eed55a3e6615_chromedriver       0x00000001042cf2c6 c048eed55a3e6615_chromedriver + 1028806
6   c048eed55a3e6615_chromedriver       0x00000001042b4d23 c048eed55a3e6615_chromedriver + 920867
7   c048eed55a3e6615_chromedriver       0x000000010427ba4b c048eed55a3e6615_chromedriver + 686667
8   c048eed55a3e6615_chromedriver       0x0000

In [49]:
data

,Título,Precio,Habitaciones,Planta,Superficie,Baños,Ascensor,Fecha Publicación,Links,Barrio,Ciudad
0,"Piso en Avenida Padre Isla, 136d, San Esteban",115.000 €,3,3,88,1,SI,23,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
1,"Piso en Avenida Mariano Andrés, 169, San Esteban",89.000 €,3,None,90,2,SI,2120,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
2,"Piso en Calle Porma, 9, San Esteban",95.000 €,2,None,70,2,SI,352,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
3,"Dúplex en Avenida de Mariano Andrés, San Esteban",180.000 €,3,None,102,2,SI,217,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
4,"Piso en Avenida Álvaro López Núñez, San Esteban",113.500 €,3,1,100,2,SI,10,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
...,...,...,...,...,...,...,...,...,...,...,...
24,Casa o chalet con 1 baño en San Esteban,80.000 €,6,None,172,1,NO,133,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
25,"Piso en Calle Catoute, 8, San Esteban",95.000 €,3,None,120,1,SI,535,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
26,Piso con terraza en San Esteban,230.000 €,4,3,145,2,NO,2,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
27,Casa o chalet con 4 baños en San Esteban,695.000 €,6,0,892,4,NO,78,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon


In [50]:
import pandas as pd

In [128]:
data.to_csv('for_sale.csv')